In [1]:
# Load required libraries
library(tidyverse)
library(janitor)
library(dplyr)
library(ggplot2)
library(skimr)
library(purrr)
library(lubridate)

# Source helper scripts
source("../../R/apply_factors.R")
source("../../R/analysis_helpers.R")
source("../../R/temporal_helpers.R")

# Load data
tables <- list(
  Orders  = readr::read_csv("../../data/processed/Orders.csv"),
  Returns = readr::read_csv("../../data/processed/Returns.csv"),
  People  = readr::read_csv("../../data/processed/People.csv")
)

# Apply factor transformations
tables <- apply_factors(tables)

# Extract tables
orders  <- tables$Orders
returns <- tables$Returns
people  <- tables$People

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Rows: 51290 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): order_id, ship_mode, customer_name, segment, state, country, mark...
dbl   (6): sales, quantity, discount, profit, shipping_cost, year
date  (2): order_date, ship_date

ℹ Use `spec()` to retrieve the full column specification f

## Returned vs Non-returned Profit per Order

This analysis evaluates whether returned orders differ systematically in profitability compared to non-returned orders. Let \( R_i \in \{0,1\} \) indicate whether order \( i \) was returned.

We test for differences in mean profit between the two groups:

$$
H_0: \mathbb{E}[\text{Profit}_i \mid R_i = 1] = \mathbb{E}[\text{Profit}_i \mid R_i = 0]
$$
$$
H_A: \mathbb{E}[\text{Profit}_i \mid R_i = 1] \neq \mathbb{E}[\text{Profit}_i \mid R_i = 0]
$$

Because the two groups may exhibit unequal variances, we apply a Welch two-sample t-test rather than a pooled-variance test.

In [2]:
orders <- orders %>%
  left_join(
    returns %>% mutate(returned = 1L),
    by = "order_id",
    relationship = "many-to-many"
  ) %>%
  mutate(returned = ifelse(is.na(returned), 0L, returned))

t.test(profit ~ returned, data = orders)


	Welch Two Sample t-test

data:  profit by returned
t = -2.8266, df = 3307.3, p-value = 0.004733
alternative hypothesis: true difference in means between group 0 and group 1 is not equal to 0
95 percent confidence interval:
 -18.910110  -3.420488
sample estimates:
mean in group 0 mean in group 1 
        27.9759         39.1412 


## Discount vs Return Probability

To assess whether discounting increases return risk, return incidence is modeled as a binary outcome using logistic regression. Let \( D_i \) denote the discount applied to order \( i \).

The model is specified as:

$$
\Pr(R_i = 1) = \text{logit}^{-1}(\beta_0 + \beta_1 D_i)
$$

where $ \text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}} $.

A positive and statistically significant estimate of \( \beta_1 \) indicates that higher discounts are associated with a higher probability of return.

In [3]:
discount_return_logit <- glm(
  returned ~ discount,
  data = orders,
  family = binomial(link = "logit")
)

summary(discount_return_logit)


Call:
glm(formula = returned ~ discount, family = binomial(link = "logit"), 
    data = orders)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -2.65785    0.02178 -122.02  < 2e-16 ***
discount    -0.79281    0.09824   -8.07 7.03e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 23159  on 51294  degrees of freedom
Residual deviance: 23089  on 51293  degrees of freedom
AIC: 23093

Number of Fisher Scoring iterations: 5


## Return Rate by Discount Bucket

To validate the discount–return relationship without imposing a linear or parametric structure, discounts are discretized into ordered buckets.

Let $ B_i $ denote the discount bucket for order $ i $. Return incidence is summarized in a contingency table and tested using Pearson’s chi-squared test:

$$
H_0: R_i \perp B_i
$$

Rejection of the null hypothesis indicates that return probability varies systematically across discount levels, providing nonparametric confirmation of the regression results.

In [4]:
orders <- orders %>%
  mutate(
    discount_bucket = cut(
      discount,
      breaks = c(0, 0.1, 0.3, 0.5, 1),
      include.lowest = TRUE,
      labels = c("Low", "Medium", "High", "Very High")
    )
  )

discount_return_table <- table(orders$returned, orders$discount_bucket)
chisq.test(discount_return_table)



	Pearson's Chi-squared test

data:  discount_return_table
X-squared = 141.18, df = 3, p-value < 2.2e-16


#  Discount Buckets and Profitability

This analysis examines whether discount depth affects average profitability independently of return behavior. Let $ \Pi_i $ denote profit and $ B_i $ the discount bucket for order $ i $.

A one-way ANOVA is used to test:

$$
H_0: \mathbb{E}[\Pi_i \mid B_i = b] \text{ is equal for all } b
$$

The F-statistic evaluates whether between-bucket variation in profit exceeds within-bucket variation, indicating systematic differences in profitability across discount levels.


In [5]:
anova_profit_discount <- aov(
  profit ~ discount_bucket,
  data = orders
)

summary(anova_profit_discount)

                   Df    Sum Sq  Mean Sq F value Pr(>F)    
discount_bucket     3 1.576e+08 52529559    1921 <2e-16 ***
Residuals       51291 1.403e+09    27351                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

# Return Rate vs Product Margin

Finally, we test whether structurally low-margin products exhibit higher return risk. Product margin is defined as:

$$
\text{Margin}_i = \frac{\text{Profit}_i}{\text{Sales}_i}
$$

Return probability is modeled as a function of margin using logistic regression:

$$
\Pr(R_i = 1) = \text{logit}^{-1}(\beta_0 + \beta_1 \text{Margin}_i)
$$

A negative estimate of $ \beta_1 $ indicates that lower-margin products are more likely to be returned, linking return behavior directly to product-level economic efficiency.

In [6]:
orders <- orders %>%
  mutate(margin = profit / sales) %>%
  filter(is.finite(margin))

margin_return_logit <- glm(
  returned ~ margin,
  data = orders,
  family = binomial(link = "logit")
)

summary(margin_return_logit)


Call:
glm(formula = returned ~ margin, family = binomial(link = "logit"), 
    data = orders)

Coefficients:
            Estimate Std. Error  z value Pr(>|z|)    
(Intercept) -2.79982    0.01963 -142.662   <2e-16 ***
margin       0.46656    0.04997    9.336   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 23159  on 51294  degrees of freedom
Residual deviance: 23058  on 51293  degrees of freedom
AIC: 23062

Number of Fisher Scoring iterations: 6
